In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Creación de datos temporales simulados

In [ ]:
# Crear serie temporal de signos vitales (mediciones cada minuto durante 6 horas)
np.random.seed(42)
n_mediciones = 360  # 6 horas * 60 minutos

# Generar timestamps
inicio = pd.Timestamp('2025-01-01 00:00:00')
timestamps = pd.date_range(start=inicio, periods=n_mediciones, freq='1min')

# Generar datos con tendencias y ruido
df = pd.DataFrame({
    'timestamp': timestamps,
    'HR': 75 + np.sin(np.arange(n_mediciones) * 0.05) * 10 + np.random.normal(0, 3, n_mediciones),
    'SBP': 120 + np.sin(np.arange(n_mediciones) * 0.03) * 15 + np.random.normal(0, 5, n_mediciones),
    'DBP': 80 + np.sin(np.arange(n_mediciones) * 0.03) * 10 + np.random.normal(0, 3, n_mediciones),
    'SpO2': 97 + np.random.normal(0, 1, n_mediciones)
})

# Añadir algunos picos anómalos
df.loc[100, 'HR'] = 140  # Pico súbito
df.loc[200, 'SBP'] = 170
df.loc[250, 'HR'] = 45

df.set_index('timestamp', inplace=True)

print("Primeras mediciones:")
print(df.head(10))
print(f"\nTotal de mediciones: {len(df)}")
print(f"Rango temporal: {df.index[0]} a {df.index[-1]}")

## 1. Media móvil (Rolling Mean) - Ventana de 1 hora

In [ ]:
# Ventana de 60 minutos para suavizar ruido
ventana_1h = 60

# Calcular media móvil para HR
df['HR_rolling_mean'] = df['HR'].rolling(window=ventana_1h).mean()

# Calcular media móvil para SBP
df['SBP_rolling_mean'] = df['SBP'].rolling(window=ventana_1h).mean()

print("Datos con media móvil (ventana 1 hora):")
print(df[['HR', 'HR_rolling_mean', 'SBP', 'SBP_rolling_mean']].head(70))

## 2. Desviación estándar móvil - Detección de variabilidad

In [ ]:
# Calcular desviación estándar móvil
df['HR_rolling_std'] = df['HR'].rolling(window=ventana_1h).std()
df['SBP_rolling_std'] = df['SBP'].rolling(window=ventana_1h).std()

print("Desviación estándar móvil:")
print(df[['HR', 'HR_rolling_std', 'SBP', 'SBP_rolling_std']].head(70))

# Identificar periodos de alta variabilidad
umbral_variabilidad = df['HR_rolling_std'].mean() + 2 * df['HR_rolling_std'].std()
periodos_inestables = df[df['HR_rolling_std'] > umbral_variabilidad]

print(f"\n=== Periodos de alta variabilidad (HR) ===")
print(f"Umbral: {umbral_variabilidad:.2f}")
print(f"Periodos detectados: {len(periodos_inestables)}")
if len(periodos_inestables) > 0:
    print(periodos_inestables[['HR', 'HR_rolling_std']])

## 3. Máximos y mínimos móviles - Detección de picos

In [ ]:
# Calcular máximo y mínimo en ventana móvil
df['HR_rolling_max'] = df['HR'].rolling(window=ventana_1h).max()
df['HR_rolling_min'] = df['HR'].rolling(window=ventana_1h).min()

df['SBP_rolling_max'] = df['SBP'].rolling(window=ventana_1h).max()
df['SBP_rolling_min'] = df['SBP'].rolling(window=ventana_1h).min()

# Calcular rango móvil (diferencia entre max y min)
df['HR_rolling_range'] = df['HR_rolling_max'] - df['HR_rolling_min']
df['SBP_rolling_range'] = df['SBP_rolling_max'] - df['SBP_rolling_min']

print("Estadísticas de rango móvil:")
print(df[['HR', 'HR_rolling_min', 'HR_rolling_max', 'HR_rolling_range']].describe())

## 4. Detección de picos súbitos

In [ ]:
# Detectar picos: valores que están muy alejados de la media móvil
df['HR_desviacion_media'] = abs(df['HR'] - df['HR_rolling_mean'])

# Umbral: 2 veces la desviación estándar móvil
df['HR_es_pico'] = df['HR_desviacion_media'] > (2 * df['HR_rolling_std'])

picos_detectados = df[df['HR_es_pico'] == True]

print("=== Picos súbitos detectados en HR ===")
print(f"Total de picos: {len(picos_detectados)}")
print("\nDetalle de los picos:")
print(picos_detectados[['HR', 'HR_rolling_mean', 'HR_desviacion_media', 'HR_rolling_std']])

## 5. Tendencia de presión arterial - Análisis de 1 hora

In [ ]:
# Analizar tendencia de presión en periodo específico (primera hora)
primera_hora = df.iloc[:60]

print("=== Análisis de tendencia: Primera hora ===")
print(f"SBP inicial: {primera_hora['SBP'].iloc[0]:.1f} mmHg")
print(f"SBP final: {primera_hora['SBP'].iloc[-1]:.1f} mmHg")
print(f"SBP promedio: {primera_hora['SBP'].mean():.1f} mmHg")
print(f"SBP máxima: {primera_hora['SBP'].max():.1f} mmHg")
print(f"SBP mínima: {primera_hora['SBP'].min():.1f} mmHg")
print(f"Variabilidad (std): {primera_hora['SBP'].std():.1f} mmHg")

## 6. Visualización de ventanas móviles

In [ ]:
# Visualizar primeras 3 horas para mejor claridad
df_plot = df.iloc[:180]

fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Gráfico 1: HR con media móvil y bandas de desviación
axes[0].plot(df_plot.index, df_plot['HR'], label='HR original', alpha=0.5, color='blue')
axes[0].plot(df_plot.index, df_plot['HR_rolling_mean'], label='Media móvil (1h)', 
             color='red', linewidth=2)
axes[0].fill_between(df_plot.index, 
                       df_plot['HR_rolling_mean'] - df_plot['HR_rolling_std'],
                       df_plot['HR_rolling_mean'] + df_plot['HR_rolling_std'],
                       alpha=0.2, color='red', label='±1 std')
axes[0].scatter(picos_detectados.index, picos_detectados['HR'], 
                color='orange', s=100, zorder=5, label='Picos detectados')
axes[0].set_ylabel('Heart Rate (bpm)')
axes[0].set_title('Heart Rate: Ventana móvil y detección de picos')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Gráfico 2: SBP con rango móvil
axes[1].plot(df_plot.index, df_plot['SBP'], label='SBP original', alpha=0.5, color='green')
axes[1].plot(df_plot.index, df_plot['SBP_rolling_mean'], label='Media móvil (1h)', 
             color='darkgreen', linewidth=2)
axes[1].plot(df_plot.index, df_plot['SBP_rolling_max'], label='Máximo móvil', 
             color='red', linestyle='--', alpha=0.7)
axes[1].plot(df_plot.index, df_plot['SBP_rolling_min'], label='Mínimo móvil', 
             color='blue', linestyle='--', alpha=0.7)
axes[1].set_ylabel('Systolic BP (mmHg)')
axes[1].set_xlabel('Tiempo')
axes[1].set_title('Presión Arterial Sistólica: Rango móvil (1 hora)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Gráficos generados exitosamente")

## 7. Diferentes tamaños de ventana

In [ ]:
# Comparar diferentes tamaños de ventana
df['HR_roll_15min'] = df['HR'].rolling(window=15).mean()
df['HR_roll_30min'] = df['HR'].rolling(window=30).mean()
df['HR_roll_60min'] = df['HR'].rolling(window=60).mean()
df['HR_roll_120min'] = df['HR'].rolling(window=120).mean()

# Visualizar efecto de diferentes ventanas
df_plot2 = df.iloc[:180]

plt.figure(figsize=(14, 6))
plt.plot(df_plot2.index, df_plot2['HR'], label='Original', alpha=0.3, color='gray')
plt.plot(df_plot2.index, df_plot2['HR_roll_15min'], label='15 min', linewidth=1.5)
plt.plot(df_plot2.index, df_plot2['HR_roll_30min'], label='30 min', linewidth=1.5)
plt.plot(df_plot2.index, df_plot2['HR_roll_60min'], label='60 min', linewidth=2)
plt.plot(df_plot2.index, df_plot2['HR_roll_120min'], label='120 min', linewidth=2)
plt.ylabel('Heart Rate (bpm)')
plt.xlabel('Tiempo')
plt.title('Comparación de diferentes tamaños de ventana móvil')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Comparación de ventanas completada")

## Resumen

### Operaciones de ventana móvil implementadas:

1. **Media móvil (`rolling().mean()`)**: Suaviza ruido y captura tendencias
2. **Desviación estándar móvil (`rolling().std()`)**: Detecta variabilidad y periodos inestables
3. **Máximos/Mínimos móviles (`rolling().max()` / `rolling().min()`)**: Identifica rangos y picos
4. **Detección de anomalías**: Combinando estadísticas móviles

### Aplicaciones clínicas:
- **Ventana de 1 hora**: Balance entre capturar tendencia y mantener responsividad
- **Picos súbitos**: Cambios rápidos que indican posible inestabilidad hemodinámica
- **Variabilidad alta**: Fluctuaciones que requieren atención clínica

### Buenas prácticas:
- Elegir tamaño de ventana según frecuencia de datos y fenómeno clínico
- Considerar valores NaN al inicio (ventanas incompletas)
- Combinar múltiples estadísticas para detección robusta de anomalías